# 3. LangChain基础

## 3.3. LangChain的回退

### 3.3.1. 处理LLM调用API的错误

In [1]:
from unittest.mock import patch
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
import httpx
from openai import RateLimitError

# 创建一个 RateLimitError 对象
response = httpx.Response(
    status_code=429,                                                       # 429: Too Many Requests 错误码
    request=httpx.Request('GET', 'https://api.openai.com/v1/completions')  # 模拟一个请求对象
)

body = {
    'error': {
        'message': 'Rate limit exceeded.',
        'type': 'rate_limit_error',
        'param': None,
        'code': 'rate_limit_error'
    }
}

error = RateLimitError(message="Rate limit exceeded.", response=response, body=body)
# print(error)

#### 1. 模拟触发速率限制错误

In [2]:
openai_api_key = 'EMPTY'
openai_api_base = 'http://localhost:11434/v1'
openai_llm = ChatOpenAI(openai_api_key=openai_api_key, openai_api_base=openai_api_base, temperature=0, max_tokens=256, max_retries=0)

with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(openai_llm.invoke("你是谁？"))
    except RateLimitError:
        print("遇到错误")

遇到错误


#### 2. 测试回退功能

In [3]:
openai_api_key = 'EMPTY'
openai_api_base = 'http://localhost:11434/v1'
openai_llm = ChatOpenAI(openai_api_key=openai_api_key, openai_api_base=openai_api_base, temperature=0, max_tokens=256, max_retries=0)
qwen_llm = ChatOllama(model="qwen:1.8b")
llm = openai_llm.with_fallbacks([qwen_llm])

with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(llm.invoke("你是谁？"))
    except RateLimitError:
        print("遇到错误")

/tmp/ipykernel_449183/3922159719.py:4: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  qwen_llm = ChatOllama(model="qwen:1.8b")


content='我是来自阿里云的大规模语言模型，我被命名为通义千问（QianWen）。\n\n通义千问是一个基于深度学习技术的多模态对话系统。它能够理解和生成各种语言，包括但不限于中文、英文、日文等。\n\n通义千问的应用非常广泛，主要在以下几个方面发挥着作用：\n\n1. 机器翻译：通义千问可以对文本进行逐字翻译，从而实现不同语言之间的无缝交流。\n\n2. 智能客服：通义千问还可以通过语音识别和自然语言处理技术，为用户提供智能的客户服务，帮助用户解决实际的问题。\n\n3. 内容创作与生成：通义千问还能够通过深度学习和强化学习等技术手段，支持用户进行内容创作和生成，如小说、诗歌、评论等。' additional_kwargs={} response_metadata={'model': 'qwen:1.8b', 'created_at': '2025-02-22T09:05:23.491088868Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 43393526551, 'load_duration': 24759140, 'prompt_eval_count': 11, 'prompt_eval_duration': 368000000, 'eval_count': 174, 'eval_duration': 42999000000} id='run-de6a0df1-29de-4db6-9a16-f9e741695616-0'


#### 3. 使用具有回退功能的LLM

In [4]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你真是一个贴心的助手，每次回复都会附上赞美之词。",
        ),
        ("human", "为什么你喜欢{city}"),
    ]
)

chain = prompt | llm
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(chain.invoke({"city":"利川"}))
    except RateLimitError:
        print("Hit error")

content='我喜欢利川，主要有以下几个原因：\n\n1. 历史文化：利川作为川东最大的城市之一，拥有丰富的历史文化遗产。如利川古城、利川森林公园等，这些都是我特别喜欢的部分。\n\n2. 食物美味：利川不仅有丰富的历史文化，还有着美食的美誉。如利川鱼粉、利川辣椒炒肉等，这些美食都是我特别喜欢的部分。\n\n3. 气候宜人：利川位于四川盆地边缘，气候宜人。全年气温在10-25摄氏度之间波动，雨量适中，空气质量优良，这些都是我特别喜欢的部分。' additional_kwargs={} response_metadata={'model': 'qwen:1.8b', 'created_at': '2025-02-22T09:06:02.542774483Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 34616850521, 'load_duration': 25016829, 'prompt_eval_count': 32, 'prompt_eval_duration': 455000000, 'eval_count': 137, 'eval_duration': 34134000000} id='run-fefde34a-066d-4e9f-b9e7-90512495e7e8-0'


### 3.3.2. 处理序列回退

In [5]:
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain.prompts import PromptTemplate, ChatPromptTemplate

# 添加一个字符串输出解释器，以便两个LLM的输出是相同类型的
from langchain_core.output_parsers import StrOutputParser

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是一个贴心的助手，每次回复都会附上赞美之词。",
        ),
        ("human", "为什么你喜欢{city}"),
    ]
)

# 使用一个错误的模型名称来构建一个错误的链
openai_api_key = 'EMPTY'
openai_api_base = 'http://localhost:11434/v1'
chat_model = ChatOpenAI(openai_api_key=openai_api_key, openai_api_base=openai_api_base, model_name="gpt-fake")
bad_chain = chat_prompt | chat_model | StrOutputParser()

# 构建一个正确的链
from langchain_core.prompts import PromptTemplate
prompt_template = """说明：你应该在回复中始终包含赞美之词。
问题：你为什么喜欢{city}？"""
prompt = PromptTemplate.from_template(prompt_template)
llm = ChatOllama(model="qwen:1.8b")
good_chain = prompt | llm

# 构建一条将两条链合并在一起的链
chain = bad_chain.with_fallbacks([good_chain])
chain.invoke({"city":"武汉"})

AIMessage(content='我之所以特别喜爱武汉，原因主要有以下几点：\n\n首先，武汉拥有独特的地理位置和历史背景。它位于中国中部地区，地处长江与汉江交汇处的三角洲地带，是连接东、西两半球的主要交通枢纽之一。\n\n其次，武汉是一座历史悠久的城市，拥有丰富的文化底蕴。从古至今，武汉都是重要的商业中心和文化名城，吸引了大量的游客和学者前来访问、学习和研究，为武汉的经济发展注入了强大的动力。\n\n综上所述，我之所以特别喜爱武汉，原因主要有独特的地理位置和历史背景、深厚的文化底蕴、重要的经济地位和发展潜力等多方面的因素共同作用的结果。', additional_kwargs={}, response_metadata={'model': 'qwen:1.8b', 'created_at': '2025-02-22T09:07:17.925432031Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 32623213385, 'load_duration': 24394783, 'prompt_eval_count': 28, 'prompt_eval_duration': 354000000, 'eval_count': 133, 'eval_duration': 32243000000}, id='run-f54a7a4a-72dd-4b26-bd5c-197e73851cb9-0')

### 3.3.3. 处理长输入

In [6]:
from langchain_community.chat_models import ChatOllama

short_llm = ChatOllama(model="deepseek-r1:1.5b")
long_llm = ChatOllama(model="deepseek-r1:8b")
llm = short_llm.with_fallbacks([long_llm])

inputs = "下一个数字是：" + ", ".join(["one", "two"] * 3000)

In [7]:
# 使用处理短输出的模型，这里没有api，本地模型没有超出token，不试了
try:
    print(short_llm.invoke(inputs))
except Exception as e:
    print(e)

content="<think>\n\n</think>\n\nIt seems like you've provided a sequence of numbers but it's cut off in your text. Could you please provide the complete sequence up to which you need an explanation? For example, if the sequence is 1, 2, 4, 8,... and so on, I can explain how this sequence progresses.\n\nOnce you provide the specific numbers from where the next one should be, I'll be able to tell you the pattern or rule that generates the sequence, and thus determine what the next number is." additional_kwargs={} response_metadata={'model': 'deepseek-r1:1.5b', 'created_at': '2025-02-22T09:08:10.394327249Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 42854650547, 'load_duration': 1579658127, 'prompt_eval_count': 2048, 'prompt_eval_duration': 16255000000, 'eval_count': 109, 'eval_duration': 25018000000} id='run-64feacde-d873-4d98-9091-efa362495873-0'


In [8]:
# 回退到长输入模型
try:
    print(llm.invoke(inputs))
except Exception as e:
    print(e)

content="<think>\n\n</think>\n\nIt seems like your message got cut off and ended abruptly. Could you please provide more details or clarify what you'd like me to do? If you're looking for a specific number in a sequence or need assistance with something else, feel free to share the details!" additional_kwargs={} response_metadata={'model': 'deepseek-r1:1.5b', 'created_at': '2025-02-22T09:08:43.966489602Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 30487077767, 'load_duration': 26267834, 'prompt_eval_count': 2048, 'prompt_eval_duration': 16629000000, 'eval_count': 58, 'eval_duration': 13829000000} id='run-36bcd4a6-7773-42c4-82e9-7a718ecaed73-0'


### 3.3.4. 回退到更好的模型

In [9]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import DatetimeOutputParser

prompt = ChatPromptTemplate.from_template(
    "what time was {event} (in %Y-%m-%dT%H:%M:%S.%fZ format - only return this value)"
)

deepseek15 = ChatOllama(model="deepseek-r1:1.5b") | DatetimeOutputParser()
deepseek80 = ChatOllama(model="deepseek-r1:8b") | DatetimeOutputParser()
only15 = prompt | deepseek15
fallback80 = prompt | deepseek15.with_fallbacks([deepseek80])

In [10]:
# 使用deepseek1.5
try:
    print(only15.invoke({"event": "the superbowl in 1994"}))
except Exception as e:
    print(f"Error: {e}")

Error: Could not parse datetime string: <think>
Alright, so I need to figure out what time the Super Bowl was held in 1994 and then convert that date into a specific datetime format. Let me break down how I can approach this.

First, I know that the Super Bowl is an annual event celebrating football sports organized by the NFL. It's usually held on a Sunday in early January or late February for most years. The user wants the time it was held in 1994, so I should check if there were any events in 1994.

I remember that the last Super Bowl before 1994 was in 1987, and the next one was in 2005. So, does that mean there wasn't a Super Bowl in 1994? I think football schedules aren't set every year; they're based on factors like injuries and game scheduling. Maybe the Super Bowl in 1994 didn't take place. But I'm not entirely sure. Let me double-check some sources or knowledge about historical Super Bowls.

Upon checking, I see that the last Super Bowl was in 2005, so it appears there wasn't

In [11]:
# 回退到deepseek80
try:
    print(fallback80.invoke({"event": "the superbowl in 1994"}))
except Exception as e:
    print(f"Error: {e}")

Error: Could not parse datetime string: <think>
Okay, so I need to figure out when the Super Bowl was in 1994. Hmm, let's start by recalling what I know about the Super Bowl dates. 

I remember that the first Super Bowl was in 1996. It happened on February 7th at 8:00 p.m. That was a big event for fans everywhere. But wait, the user is asking specifically about 1994. So I can't just say it's before that date.

I think there was another Super Bowl in 1993, but I'm not entirely sure if that one happened on December 8th at night. Let me check my memory. Yes, the 1993 Super Bowl was also a big event with a lot of coverage. It took place on December 8th at 8:00 p.m. in Las Vegas.

So putting this together, since 1994 came right after that in time, and there were no Super Bowls between 1993 and 1996, the answer must be that there was no Super Bowl in 1994.
</think>

The Super Bowl did not occur in 1994. The closest one was the 1993 Super Bowl which took place on December 8th at 8:00 p.m., an